# Étape 3 — Indicateurs Clés de Performance (ICP)

Cette étape consiste à calculer et analyser les indicateurs clés de performance à partir
des données nettoyées. Les analyses utilisent Pandas et les fonctions groupby, agg,
pivot_table, sort_values et nlargest.


In [5]:
df = pd.read_csv("order_lines_clean.csv")
df['order_date'] = pd.to_datetime(df['order_date'])
df.head()

,order_id,customer_id,product_id,order_date,quantity,unit_price,discount_pct,gross_amount,net_amount,payment_method,channel,marketing_source,delivery_days,returned,review_score,segment,city,category
0,O00001,C0201,P031,2024-06-16,4,165.54,0.18,662.16,542.97,Carte,App,Organique,1.0,0,5.0,Indépendant,Port-au-Prince,Cours
1,O00002,C0141,P041,2024-06-10,5,46.48,0.14,232.40,199.86,Carte,Web,Réseaux sociaux,0.0,0,4.0,Étudiant,Abidjan,Livre
2,O00003,C0025,P005,2024-08-29,2,1123.49,0.04,2246.98,2157.10,Virement,Web,Réseaux sociaux,3.0,0,4.0,Étudiant,Cap-Haïtien,Laptop
3,O00004,C0188,P042,2024-01-27,2,68.31,0.22,136.62,106.56,Carte,Boutique,Partenariat,0.0,0,4.0,Indépendant,Abidjan,Cloud
4,O00005,C0023,P052,2025-08-06,3,1932.53,0.17,5797.59,4812.00,Carte,Web,Référencement payant,9.0,0,5.0,Professionnel,Cap-Haïtien,Laptop


## 1. Chiffre d’affaires (CA)


In [7]:
ca_total = df['net_amount'].sum()

df['month'] = df['order_date'].dt.to_period('M')
ca_mensuel = (
    df.groupby('month')['net_amount']
      .sum()
      .sort_values(ascending=False)
)

print("Le Chiffre d'Affaire total :", ca_total)
print("Le Chiffre d'Affaire mensuel :", 
      ca_mensuel)

Le Chiffre d'Affaire total : 2023658.44
Le Chiffre d'Affaire mensuel : month
2024-02    124550.19
2025-08    111647.31
2024-08    108552.22
2025-01    107745.22
2024-01     96136.26
2024-06     95653.19
2025-10     95237.73
2025-07     94230.63
2024-12     93965.73
2025-06     89469.25
2024-07     85929.85
2024-03     85860.28
2025-05     84927.94
2024-04     82740.55
2025-04     79869.32
2025-09     79536.37
2024-05     73921.77
2025-03     72754.30
2024-09     72245.42
2025-02     71454.81
2024-10     67970.70
2024-11     66455.22
2025-11     46574.92
2025-12     36229.26
Freq: M, Name: net_amount, dtype: float64


## 2. Panier moyen (AOV)


In [3]:
nb_commandes = df['order_id'].nunique()
aov = ca_total / nb_commandes
print("Le panier moyen (AOV) : ", aov)

924.8895978062156

## 3. Taux de remise moyen


In [4]:
remise_globale = df['discount_pct'].mean() * 100

remise_par_categorie = (
    df.groupby('category')
      .agg(remise_moyenne=('discount_pct', 'mean'))
      .sort_values('remise_moyenne', ascending=False)
) * 100

print("Le taux de remise globale :", remise_globale)
print("Le taux de remise par catégorie : ", 
      remise_par_categorie)

(11.523765996343693,
             remise_moyenne
 category                  
 Laptop           11.653563
 Accessoire       11.650000
 Cours            11.640988
 Cloud            11.637931
 Livre            11.339394
 Logiciel         11.079038)

## 4. Taux de retour


In [5]:
taux_retour_global = df['returned'].mean() * 100

taux_retour = pd.pivot_table(
    df,
    values='returned',
    index='category',
    columns='channel',
    aggfunc='mean'
) * 100

print("Le taux de retour global : ", taux_retour_global)
print("Le taux de retour : ", 
      taux_retour)

(4.113345521023766,
 channel          App  Boutique       Web
 category                                
 Accessoire  4.938272  3.846154  8.270677
 Cloud       4.285714  0.000000  3.731343
 Cours       4.382470  0.000000  4.266667
 Laptop      2.222222  3.333333  6.132075
 Livre       4.716981  0.000000  3.191489
 Logiciel    4.040404  2.777778  3.205128)

## 5. Score d’avis moyen


In [6]:
score_global = df['review_score'].mean()

score_par_categorie = (
    df.groupby('category')['review_score']
      .mean()
      .sort_values(ascending=False)
)

score_par_delai = (
    df.groupby('delivery_days')['review_score']
      .mean()
      .sort_values(ascending=False)
)

top_categories = score_par_categorie.nlargest(5)

print("Le score global : ", score_global)
print("Le score par catégorie : ", 
      score_par_categorie)
print("Le score par délai : ", 
      score_par_delai)
print("Le top des catégories : ",
      top_categories)

(4.356309074643122,
 category
 Accessoire    4.423227
 Cours         4.381142
 Logiciel      4.356175
 Laptop        4.331706
 Livre         4.327505
 Cloud         4.297744
 Name: review_score, dtype: float64,
 delivery_days
 19.0    4.678723
 21.0    4.500000
 7.0     4.498920
 14.0    4.462963
 5.0     4.456320
 17.0    4.428723
 13.0    4.397031
 3.0     4.388599
 2.0     4.375322
 9.0     4.340909
 6.0     4.339295
 1.0     4.338702
 16.0    4.333333
 0.0     4.329890
 8.0     4.309818
 12.0    4.308446
 4.0     4.262766
 18.0    4.250000
 11.0    4.221565
 10.0    4.193664
 15.0    4.100000
 Name: review_score, dtype: float64,
 category
 Accessoire    4.423227
 Cours         4.381142
 Logiciel      4.356175
 Laptop        4.331706
 Livre         4.327505
 Name: review_score, dtype: float64)

In [9]:
ca_mensuel.to_csv('C:\\Documents\\Abdarare\\FDS-FRST-SDMIA\\Python\\Mini_Projet_Pandas\\Livrables\\KPI_Calculés\\kpi_ca_mensuel.csv', index=False)

## Conclusion

Les ICP calculés offrent une vision globale et segmentée de la performance commerciale,
de la politique de remise, de la qualité logistique et de la satisfaction client.
